In [1]:
import os
os.chdir('../')

In [2]:
%matplotlib inline
#%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [3]:
from copy import deepcopy
from typing import List, Tuple

from cycler import cycler
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.decomposition import PCA
import scipy.stats as stats
import torch
from torch import nn
import torch.nn.functional as F
import seml

import tqdm
tqdm.tqdm.pandas()
#plt.style.use('ggplot')

/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
from notebooks import mpl_latex

/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [5]:
#mpl_latex.enable_production_mode()

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [102]:
labels_to_plot = [
    'Vanilla GCN',
    'Vanilla GDC',
    'SVD GCN',
    'Jaccard GCN',
    'RGCN',
    'Soft Medoid GDC (T=0.5)'
]

In [12]:
dataset_map = {
    'cora_ml': r'\textbf{Cora ML}', # '\rotatebox{90}{Cora ML~\citep{Bojchevski2018}}',
    'citeseer': r'\textbf{Citeseer}', # ~\citep{McCallum2000}
    'pubmed': r'\textbf{PubMed}', # ~\citep{Sen2008}
    'ogbn-arxiv': r'\textbf{arXiv}', # ~\citep{Hu2020}
    'ogbn-products': r'\textbf{Products}', # ~\citep{Hu2020},
    'ogbn-papers100M': r'\textbf{Papers 100M}'
}
dataset_order = [dataset_map[k] for k in dataset_map.keys()]

In [208]:
attack_map = {
    'DICE': r'\textbf{DICE}',
    'GANG': r'\textbf{GANG (ours)}',
    'FGSM': r'\textbf{greedy FGSM}',
    'GreedyRBCD': r'\textbf{GR-BCD (ours)}',
    'PGD': r'\textbf{PGD}',
    'PRBCD': r'\textbf{PR-BCD (ours)}'
}
attack_order = [attack_map[k] for k in attack_map.keys()]

In [14]:
pm = r'\(\pm\)'
bpm = r'\(\boldsymbol{\pm}\)'

In [15]:
def make_max_bold(group):
    for col in list(group.columns):
        if not group[col].isna().all():
            idx = np.where(np.max(group[col].fillna("")) == group[col].fillna(""))[0]
            group.loc[group.index[idx], col] = rf'\textbf{{{group.loc[group.index[idx], col].iloc[0]}}}'\
                .replace(pm, bpm)
    return group

In [16]:
def make_second_best_underlined(group):
    for col in list(group.columns):
        if not group[col].isna().all() and np.unique(group[col].fillna("")).shape[0] > 1:
            second_max = np.unique(group[col].fillna("").apply(
                lambda v: v.replace(r'\textbf{', '').replace('}', '')
            ))[-2]
            idx = np.where(second_max == group[col].fillna(""))[0]
            group.loc[group.index[idx], col] = rf'\underline{{{group.loc[group.index[idx], col].iloc[0]}}}'
    return group

In [ ]:
def mark_best_and_second_best(group: pd.DataFrame, 
                              first_mark: str = r'\textbf',
                              second_mark: str = r'\underline',
                              dimension: int = 0) -> pd.DataFrame:
    iterable  

In [17]:
def calc_mean_and_error(values: pd.Series, seeds: pd.Series, with_error=True, decimal_places: int = 3): 
    values, seeds = values.values, seeds.values
    seeds = seeds[~np.isnan(values)]
    values = values[~np.isnan(values)]

    idx = np.unique(seeds, return_index=True)[1]
    values = values[idx]
    
    if with_error:
        return rf'{np.mean(values):.{decimal_places}f} $\pm$ {np.std(values)/len(values):.{decimal_places}f}'
    else:
        return rf'{np.mean(values):.{decimal_places}f}'

In [314]:
df_experiments = seml.get_results('rgnn_at_scale_attack_evasion_transfer',
                                  to_data_frame=True,
                                  fields=['batch_id', 'slurm', 'config', 'result'])
df_experiments = df_experiments[df_experiments.batch_id >= 8]
df_experiments.head()

/nfs/staff-ssd/geisler/anaconda3/envs/rgnn_at_scale/lib/python3.8/site-packages/seml/evaluation.py:48: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  parsed = pd.io.json.json_normalize(parsed, sep='.')


,_id,batch_id,slurm.experiments_per_job,slurm.sbatch_options.gres,slurm.sbatch_options.mem,slurm.sbatch_options.cpus-per-task,slurm.sbatch_options.time,slurm.sbatch_options.partition,slurm.sbatch_options.nodes,config.overwrite,config.db_collection,config.dataset,config.attack,config.epsilons,config.surrogate_params.n_filters,config.surrogate_params.dropout,config.surrogate_params.train_params.lr,config.surrogate_params.train_params.weight_decay,config.surrogate_params.train_params.patience,config.surrogate_params.train_params.max_epochs,config.surrogate_params.do_checkpoint,config.surrogate_params.n_chunks,config.binary_attr,config.seed,config.artifact_dir,config.pert_adj_storage_type,config.pert_attr_storage_type,config.model_storage_type,config.device,config.display_steps,config.model_label,result.results,config.attack_params.do_synchronize,config.attack_params.edge_budget,config.attack_params.edge_step_size,config.attack_params.edge_with_random_reverse,slurm.sbatch_options.job-name,slurm.sbatch_options.array,slurm.sbatch_options.output,slurm.array_id,slurm.task_id,config.attack_params.keep_heuristic,config.attack_params.search_space_size,config.attack_params.feature_dedicated_iterations,config.attack_params.feature_greedy_opt,config.attack_params.feature_lr,config.attack_params.eps
4,13,8,1,gpu:1,32G,4,1-00:00,gpu_large,1,13,rgnn_at_scale_attack_evasion_transfer,ogbn-products,PRBCD,"[0, 0.25]",64,0.5,0.01,0.001,100,3000,True,64.0,False,0,cache,attack,evasion_transfer_attack_attr,pretrained,0,10,None,[],True,NaN,NaN,NaN,rgnn_at_scale_attack_evasion_transfer_8,0-11,/nfs/homedirs/geisler/code/robust_gnns_at_scal...,6127957.0,3.0,WeightOnly,20000000.0,NaN,NaN,NaN,NaN
5,14,8,1,gpu:1,32G,4,1-00:00,gpu_large,1,14,rgnn_at_scale_attack_evasion_transfer,ogbn-products,PRBCD,"[0, 0.25]",64,0.5,0.01,0.001,100,3000,True,64.0,False,1,cache,attack,evasion_transfer_attack_attr,pretrained,0,10,None,"[{'label': 'Vanilla GCN', 'epsilon': 0, 'accur...",True,NaN,NaN,NaN,rgnn_at_scale_attack_evasion_transfer_8,0-11,/nfs/homedirs/geisler/code/robust_gnns_at_scal...,6127957.0,4.0,WeightOnly,20000000.0,NaN,NaN,NaN,NaN
6,15,8,1,gpu:1,32G,4,1-00:00,gpu_large,1,15,rgnn_at_scale_attack_evasion_transfer,ogbn-products,PRBCD,"[0, 0.25]",64,0.5,0.01,0.001,100,3000,True,64.0,False,5,cache,attack,evasion_transfer_attack_attr,pretrained,0,10,None,[],True,NaN,NaN,NaN,rgnn_at_scale_attack_evasion_transfer_8,0-11,/nfs/homedirs/geisler/code/robust_gnns_at_scal...,6127957.0,5.0,WeightOnly,20000000.0,NaN,NaN,NaN,NaN
7,19,8,1,gpu:1,32G,4,1-00:00,gpu_large,1,19,rgnn_at_scale_attack_evasion_transfer,ogbn-products,DICE,"[0, 0.25]",64,0.5,0.01,0.001,100,3000,True,64.0,False,0,cache,attack,evasion_transfer_attack_attr,pretrained,0,10,None,[],NaN,NaN,NaN,NaN,rgnn_at_scale_attack_evasion_transfer_8,0-11,/nfs/homedirs/geisler/code/robust_gnns_at_scal...,6127957.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
8,20,8,1,gpu:1,32G,4,1-00:00,gpu_large,1,20,rgnn_at_scale_attack_evasion_transfer,ogbn-products,DICE,"[0, 0.25]",64,0.5,0.01,0.001,100,3000,True,64.0,False,1,cache,attack,evasion_transfer_attack_attr,pretrained,0,10,None,"[{'label': 'Vanilla GCN', 'epsilon': 0, 'accur...",NaN,NaN,NaN,NaN,rgnn_at_scale_attack_evasion_transfer_8,0-11,/nfs/homedirs/geisler/code/robust_gnns_at_scal...,6127957.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN


In [315]:
df_experiments.columns

Index(['_id', 'batch_id', 'slurm.experiments_per_job',
       'slurm.sbatch_options.gres', 'slurm.sbatch_options.mem',
       'slurm.sbatch_options.cpus-per-task', 'slurm.sbatch_options.time',
       'slurm.sbatch_options.partition', 'slurm.sbatch_options.nodes',
       'config.overwrite', 'config.db_collection', 'config.dataset',
       'config.attack', 'config.epsilons', 'config.surrogate_params.n_filters',
       'config.surrogate_params.dropout',
       'config.surrogate_params.train_params.lr',
       'config.surrogate_params.train_params.weight_decay',
       'config.surrogate_params.train_params.patience',
       'config.surrogate_params.train_params.max_epochs',
       'config.surrogate_params.do_checkpoint',
       'config.surrogate_params.n_chunks', 'config.binary_attr', 'config.seed',
       'config.artifact_dir', 'config.pert_adj_storage_type',
       'config.pert_attr_storage_type', 'config.model_storage_type',
       'config.device', 'config.display_steps', 'config.model_

In [316]:
[c for c in df_result.columns if c != 'batch_id']

['label', 'epsilon', 'accuracy', 'dataset', 'attack', 'seed']

In [317]:
df_experiments.shape

(248, 47)

In [318]:
df_results = [
    pd.DataFrame(r) 
    for r in df_experiments['result.results']
]
for df_result, (_, df_experiment) in zip(df_results, df_experiments.iterrows()):
    df_result['dataset'] = df_experiment['config.dataset']
    df_result['attack'] = df_experiment['config.attack']
    df_result['seed'] = df_experiment['config.seed']
    df_result['batch_id'] = df_experiment['batch_id']

df_results = pd.concat(df_results, ignore_index=True)
df_results = df_results.sort_values('batch_id')
df_results = df_results.drop_duplicates([
    c for c in df_results.columns if c != 'batch_id' and c != 'accuracy'
], keep='last')

df_results

,dataset,attack,seed,batch_id,label,epsilon,accuracy
2055,citeseer,DICE,0,24,Vanilla GDC,0.05,0.693048
2054,citeseer,DICE,0,24,Vanilla GDC,0.01,0.702674
2053,citeseer,DICE,0,24,Vanilla GDC,0.00,0.706417
2052,citeseer,DICE,0,24,Vanilla GCN,0.25,0.675936
2051,citeseer,DICE,0,24,Vanilla GCN,0.10,0.697861
...,...,...,...,...,...,...,...
3985,ogbn-arxiv,PRBCD,0,42,Vanilla GDC,0.05,0.439129
3986,ogbn-arxiv,PRBCD,0,42,Vanilla GDC,0.10,0.387877
3987,ogbn-arxiv,PRBCD,0,42,Vanilla GDC,0.25,0.305454
3989,ogbn-arxiv,PRBCD,1,42,Vanilla GCN,0.01,0.623830


In [319]:
df_results.attack.unique()

array(['DICE', 'GreedyRBCD', 'FGSM', 'PGD', 'PRBCD', 'GANG'], dtype=object)

In [320]:
architecture_c = r'\textbf{Architecture}'
dataset_c = r'   '
attack_c = r'\textbf{Attack}'
epsilons_c = r'Frac. edges \(\boldsymbol{\epsilon}\)'

epsilons = [0.05, 0.1, 0.25]
epsilon_marks = [r'\textit', r'\underline', r'\textbf']

In [321]:
def transform_label(label: str):
    return label.replace('\n', ' ').replace(' (T=0.5)','')

In [322]:
df = []

for (dataset, attack, label, epsilon), df_group in df_results.groupby(['dataset', 'attack', 'label', 'epsilon']):
    if len(df_group.seed.unique()) != 3:
        print(f'For {dataset}-{attack}-{epsilon} collected runs for seed {df_group.seed.tolist()}')

    accurcy = calc_mean_and_error(df_group.accuracy, df_group.seed, with_error=False)
    df.append({
        dataset_c: dataset_map[dataset],
        architecture_c: transform_label(label),
        attack_c: attack_map[attack],
        epsilons_c: epsilon,
        'accuracy': accurcy
    })
    
df = pd.DataFrame(df)
df

,,\textbf{Architecture},\textbf{Attack},Frac. edges \(\boldsymbol{\epsilon}\),accuracy
0,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.00,0.714
1,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.01,0.713
2,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.05,0.708
3,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.10,0.700
4,\textbf{Citeseer},Jaccard GCN,\textbf{DICE},0.25,0.677
...,...,...,...,...,...
680,\textbf{PubMed},Vanilla GDC,\textbf{PR-BCD (ours)},0.00,0.776
681,\textbf{PubMed},Vanilla GDC,\textbf{PR-BCD (ours)},0.01,0.763
682,\textbf{PubMed},Vanilla GDC,\textbf{PR-BCD (ours)},0.05,0.712
683,\textbf{PubMed},Vanilla GDC,\textbf{PR-BCD (ours)},0.10,0.661


In [323]:
df = df[
    ~((df[attack_c] == attack_map['PGD']) & (df[dataset_c] == dataset_map['pubmed']))
    & ~((df[attack_c] == attack_map['FGSM']) & (df[dataset_c] == dataset_map['pubmed']))
    & ~((df[architecture_c] == transform_label('Soft Medoid GDC (T=0.5)')) & (df[dataset_c] == dataset_map['pubmed']))
]

In [324]:
df_accuracy = df[(df[epsilons_c] == 0) & (df[attack_c] == attack_map['DICE'])]
df_accuracy = df_accuracy.drop(columns=[attack_c, epsilons_c])
df_accuracy = df_accuracy.set_index([dataset_c, architecture_c])
df_accuracy.columns = pd.MultiIndex.from_product([['\textbf{Accuracy}'], [' ']])
df_accuracy

\textbf{Accuracy}
                                                           
                  \textbf{Architecture}                    
\textbf{Citeseer} Jaccard GCN                         0.714
                  RGCN                                0.646
                  SVD GCN                             0.641
                  Soft Medoid GDC (T=0.2)             0.697
                  Soft Medoid GDC                     0.707
                  Soft Medoid GDC (T=1.0)             0.712
                  Vanilla GCN                         0.712
                  Vanilla GDC                         0.709
\textbf{Cora ML}  Jaccard GCN                         0.819
                  RGCN                                0.800
                  SVD GCN                             0.761
                  Soft Medoid GDC (T=0.2)             0.793
                  Soft Medoid GDC                     0.817
                  Soft Medoid GDC (T=1.0)             0.827
                  Vanilla GCN                         0.825
                  Vanilla GDC                         0.831
\textbf{arXiv}    Vanilla GCN                         0.705
                  Vanilla GDC                         0.618
\textbf{Products} Vanilla GCN                         0.719
                  Vanilla GDC                         0.709
\textbf{PubMed}   Soft Medoid GDC (T=0.2)             0.758
                  Soft Medoid GDC (T=1.0)             0.772
                  Vanilla GCN                         0.774
                  Vanilla GDC                         0.776

In [325]:
df = df[df[epsilons_c].isin(epsilons)].copy()

In [326]:
from functools import partial

def _mark_best_and_second_best(vector: np.ndarray, 
                               first_mark: str = r'\textbf',
                               second_mark: str = r'\underline',
                               is_higher_better : bool = True) -> np.ndarray:
    vector = vector.astype(object)
    values = np.unique(vector[vector == vector])
    if not len(values):
        return vector
    
    if first_mark:
        if is_higher_better:
            mask = [vector == values[-1]]
        else:
            mask = [vector == values[0]]
        vector[mask] = np.char.add(np.char.add(first_mark + '{', vector[mask]), '}')
    if second_mark:
        if is_higher_better:
            mask = [vector == values[-2]]
        else:
            mask = [vector == values[1]]
        vector[mask] = np.char.add(np.char.add(second_mark + '{', vector[mask]), '}')
    return vector
    

def mark_best_and_second_best(df: pd.DataFrame, 
                              first_mark: str = r'\textbf',
                              second_mark: str = r'\underline',
                              axis: int = 0,
                              is_higher_better : bool = True) -> pd.DataFrame:
    df[:] = np.apply_along_axis(
        partial(_mark_best_and_second_best, first_mark=first_mark, 
                second_mark=second_mark, is_higher_better=is_higher_better),
        axis=axis,
        arr=df.values
    )
    return df

In [328]:
df[architecture_c] = df[architecture_c].astype("category")
df[architecture_c].cat.set_categories(
    [transform_label(l) for l in labels_to_plot],
    inplace=True
)
df[dataset_c] = df[dataset_c].astype("category")
df[dataset_c].cat.set_categories(
    dataset_order,
    inplace=True
)

df_piv = pd.pivot_table(
    df, 
    index=[dataset_c, architecture_c], 
    columns=[attack_c, epsilons_c], #['type', attack_c, ' '], 
    values='accuracy',
    aggfunc=lambda x: ' '.join(x)
)

df_piv = df_piv[[c for attack in attack_order for c in df_piv.columns if c[0] == attack]]

for epsilon, mark in zip(epsilons, epsilon_marks):
    current_columns = [c for c in df_piv.columns if c[1] == epsilon]
    df_piv[current_columns] = df_piv[current_columns].groupby(dataset_c).apply(
        partial(mark_best_and_second_best, first_mark=mark, second_mark='', axis=1, is_higher_better=False)
    )

def acc_to_mean_and_error(group: pd.DataFrame, with_error=True, decimal_places: int = 3):
    return calc_mean_and_error(group['accuracy'], group['seed'], with_error=with_error, decimal_places=decimal_places)

df_piv = df_piv.join(df_accuracy)
df_piv = df_piv.fillna('-')
df_piv

<ipython-input-326-6cca4f51faed>:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  vector[mask] = np.char.add(np.char.add(first_mark + '{', vector[mask]), '}')


\textbf{Attack}                         \textbf{DICE}                \
Frac. edges \(\boldsymbol{\epsilon}\)            0.05    0.1   0.25   
                  \textbf{Architecture}                               
\textbf{Cora ML}  Vanilla GCN                   0.813  0.806  0.766   
                  Vanilla GDC                   0.821  0.815  0.777   
                  SVD GCN                       0.749  0.736  0.682   
                  Jaccard GCN                   0.808  0.802  0.770   
                  RGCN                          0.791  0.785  0.756   
                  Soft Medoid GDC               0.813  0.808  0.794   
\textbf{Citeseer} Vanilla GCN                   0.702  0.692  0.670   
                  Vanilla GDC                   0.694  0.684  0.657   
                  SVD GCN                       0.630  0.615  0.573   
                  Jaccard GCN                   0.708  0.700  0.677   
                  RGCN                          0.634  0.622  0.596   
                  Soft Medoid GDC               0.704  0.701  0.693   
\textbf{PubMed}   Vanilla GCN                   0.757  0.747  0.708   
                  Vanilla GDC                   0.759  0.749  0.709   
\textbf{arXiv}    Vanilla GCN                   0.689  0.669  0.609   
                  Vanilla GDC                   0.573  0.538  0.474   
\textbf{Products} Vanilla GCN                   0.674  0.636  0.545   
                  Vanilla GDC                   0.677  0.657  0.618   

\textbf{Attack}                         \textbf{GANG (ours)}  \
Frac. edges \(\boldsymbol{\epsilon}\)                   0.05   
                  \textbf{Architecture}                        
\textbf{Cora ML}  Vanilla GCN                          0.766   
                  Vanilla GDC                          0.788   
                  SVD GCN                              0.764   
                  Jaccard GCN                          0.788   
                  RGCN                        \textit{0.701}   
                  Soft Medoid GDC             \textit{0.769}   
\textbf{Citeseer} Vanilla GCN                          0.675   
                  Vanilla GDC                          0.686   
                  SVD GCN                              0.627   
                  Jaccard GCN                          0.705   
                  RGCN                                 0.624   
                  Soft Medoid GDC                      0.704   
\textbf{PubMed}   Vanilla GCN                 \textit{0.702}   
                  Vanilla GDC                          0.727   
\textbf{arXiv}    Vanilla GCN                          0.516   
                  Vanilla GDC                          0.609   
\textbf{Products} Vanilla GCN                          0.659   
                  Vanilla GDC                          0.700   

\textbf{Attack}                                                             \
Frac. edges \(\boldsymbol{\epsilon}\)                  0.1            0.25   
                  \textbf{Architecture}                                      
\textbf{Cora ML}  Vanilla GCN                        0.732           0.658   
                  Vanilla GDC                        0.762           0.712   
                  SVD GCN                            0.760           0.722   
                  Jaccard GCN                        0.768           0.737   
                  RGCN                   \underline{0.674}           0.603   
                  Soft Medoid GDC                    0.765           0.755   
\textbf{Citeseer} Vanilla GCN                        0.644           0.593   
                  Vanilla GDC                        0.662           0.630   
                  SVD GCN                \underline{0.420}           0.539   
                  Jaccard GCN                        0.698           0.691   
                  RGCN                               0.601           0.543   
                  Soft Medoid GDC                    0.700           0.

In [330]:
print(df_piv\
    .sort_index(level=1, ascending=False, sort_remaining=False)\
    .sort_index(level=0, ascending=True, sort_remaining=True)\
    .to_latex(
        float_format='%.2f',
        caption=r'Perturbed accuracy for the proposed attacks (see Sections~\ref{sec:attackkdd}-\ref{sec:prbcd}) and baselines on all datasets (see Table~\ref{tab:datasets}). \(\epsilon\) denotes the fraction of edges perturbed (relative to the clean graph). The last column contains the clean accuracy. As this a work-in-progress report, the experiments for the defenses on the large datasets are due and on Products we did not optimize the hyperparameters for GANG. For each architecture we italicize the strongest attack where \(\epsilon=0.05\), underline where \(\epsilon=0.1\), and embolden where \(\epsilon=0.25\). From an attack perspective, a lower perturbed accuracy is better. We rerun the experiments with three different seeds. For OGB we use the provided data splits and otherwise we use random split with 20 nodes per class.', 
        label='tab:global',
        escape=False,
        multirow=True,
        multicolumn=True,
        column_format='llccccccccccccccccccc'
    )
)

\begin{table}
\centering
\caption{Perturbed accuracy for the proposed attacks (see Sections~\ref{sec:attackkdd}-\ref{sec:prbcd}) and baselines on all datasets (see Table~\ref{tab:datasets}). \(\epsilon\) denotes the fraction of edges perturbed (relative to the clean graph). The last column contains the clean accuracy. As this a work-in-progress report, the experiments for the defenses on the large datasets are due and on Products we did not optimize the hyperparameters for GANG. For each architecture we italicize the strongest attack where \(\epsilon=0.05\), underline where \(\epsilon=0.1\), and embolden where \(\epsilon=0.25\). From an attack perspective, a lower perturbed accuracy is better. We rerun the experiments with three different seeds. For OGB we use the provided data splits and otherwise we use random split with 20 nodes per class.}
\label{tab:global}
\begin{tabular}{llccccccccccccccccccc}
\toprule
                  & \textbf{Attack} & \multicolumn{3}{l}{\textbf{DICE}} & \mu